# Симуляция ткани (шарики с жёсткими пружинами) с гравитацией в 3D

Перед тем как начать хоть что-то симулировать, хотелось-бы получить базовые физические точки в пространстве. Этим и займёмся!


In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from matplotlib import rc
rc('animation', html='jshtml')

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 20000

Для начала заведём глобальных констант. Это секретные инструменты которые пригодятся нам потом.

In [ ]:
GLOBAL_CONSTANT_FORCE = np.array([0, 0, -9.5], dtype=np.float64)  # Гравитация

DT = 1 / 60                 # шаг симуляции в секундах
K = 200                     # упругость пружины
M = 0.2                     # масса точек
D = 30                      # коэфицент демпинга (СМ. РЕАЛИЗАЦИЮ НИЖЕ)
C = 60                      # альтернативное решение для демпинга

In [ ]:
# Материальная точка. В ней пока только позиция и скорость, и констрейны коробки
class MaterialPoint:
    position = np.array([0, 0, 0], dtype=np.float64)
    velocity = np.array([0, 0, 0], dtype=np.float64)
    mass = 1.0

    def __init__(self, position, mass=1.0, velocity=None):
        if velocity is None:
            velocity = np.array([0, 0, 0], dtype=np.float64)

        self.position = position
        self.mass = mass
        self.velocity = velocity

    def update(self, dt, external_forces = None):
        """
        external_forces(float[3]) -> float[3] or None
        Описывает внешние силы влияющие на систему
        Например lambda pos: GLOBAL_CONSTANT_FORCE опишет чистую гравитацию
        """
        pass

In [ ]:
# Реализация Верлета. Классовое представление здесь осталось как легаси
# из примера где я делал Эйлера, Полуейлера и Обратного Эйлера
# Пример перекочевал в следующую домашку. Ждите с нетерпением)
# Реализует нижнюю функцию + дампинг
# new_position = 2*old_position - prev_position + acceleration * dt^2
class MaterialPointVerlet(MaterialPoint):
    old_position = np.array([0, 0, 0], dtype=np.float64)

    def __init__(self, position, mass=1.0, velocity=None):
        if velocity is None:
            self.old_position = position.copy()
        else:
            self.old_position = position.copy() - velocity.copy() * DT
        super().__init__(position, mass, velocity)

    def update(self, dt, external_forces = None):
        if external_forces:
            acceleration = (external_forces(self.position, self.mass)) / self.mass
        else:
            acceleration = np.array([0, 0, 0], dtype=np.float64)

        position = self.position.copy()
        """
        Общая формула
        x[k+1] = 2x[k] - x[k-1] + (- w^2 * x[k] - dw(x[k]-x[k-1]))Δt^2
        (формула получена из неявного Ейлера с матрицой дэмпинга)
        Помним что d = c/2sqrt(km), w = sqrt(k/w)
        получаем
        x[k+1] = 2x[k] - x[k-1] + (-k/m * x[k] - c/2m*(x[k]-x[k-1]))Δt^2

        -k/m * x[k] высчитаем для каждой пружины (и для внешних сил)
        отдельно и запишем в acceleration
        2x[k] - x[k-1] + (-c/2m*(x[k]-x[k-1]))Δt^2 распишем как
        x[k] + (x[k] - x[k-1])(1 - c/2mΔt^2)
        Сюда уже подставим константу С заданную выше
        """
        self.position += (1 - C / self.mass / 2 * DT * DT) *\
                  (self.position - self.old_position) + acceleration * DT * DT
        self.old_position = position
        super().update(dt)

Просто материальной точки нам совершенно недостаточно! Добавим "связи"

*Note: использование ООП очень сильно увеличивает константу работы алгоритмов. Это могло быть серьёзной проблемой, если бы не ещё большая константа которую привносит ужастно неоптимальный рендер matplotlib. В идеальном мире вместо объектов мы бы всё хранили в том-же numpy и запускали бы вычисления на батчах, экономя время. А ещё лучше сделали бы реализацию на с++. Но это в идеальном мире.*

In [ ]:
# Реализация Верлета с дампингом.
class MaterialPointVerletWithSpring(MaterialPointVerlet):
    old_position = np.array([0, 0, 0], dtype=np.float64)
    fixed = False
    connections = []
    frame = 0

    # В симуляциях нам будут полезны "фиксированные" в пространстве точки
    def __init__(self, position, mass=1.0, velocity=None, fixed=False):
        self.connections = []
        self.fixed = fixed
        super().__init__(position, mass, velocity)

    def update(self, dt, external_forces = None):
        if self.fixed:
            return

        if external_forces:
            forces = lambda pos, mass: external_forces(pos, mass) + self.get_connection_forces()
        else:
            forces = lambda pos, mass: self.get_connection_forces()

        # Мы закинем во внешние силы все силы которые влияют на нас из пружин
        super().update(dt, forces)
        # В этом примере силы мы вытащим через get_forces() и применим вместе с гравитацией
        acceleration = (GLOBAL_CONSTANT_FORCE + self.get_connection_forces()) / self.mass


    def connect(self, other, k=1):
        distance = np.linalg.norm(self.position - other.position)
        self.connections.append({"other": other, "k": k, "distance": distance})
        other.connections.append({"other": self, "k": k, "distance": distance})


    def get_connection_forces(self):
        # Обещанный расчёт сумм F = -xk
        forces = np.array([0, 0, 0], dtype=np.float64)
        for c in self.connections:
            other = c["other"]
            other_pos = other.position
            if other.frame > self.frame:
                other_pos = other.old_position

            direction = other_pos - self.position
            offset = direction * (1 - c["distance"] / np.linalg.norm(direction)) / 2

            forces += offset * c["k"]

        return forces

Заведём несколько вспомогательных функций для построения такней и отрисовки анимации

In [ ]:
def create_rect_cloth_w(point_count_w, point_count_h, MaterialPointClass):
    """
    Вертикально висящая ткань, подвешанная с обоих сторон за point_count_w/3 вершин
    """
    x_ = np.linspace(0.25, 0.75, point_count_w)
    y_ = np.linspace(0.5, 0.5, 1)
    z_ = np.linspace(1., 0.50, point_count_h)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    positions = np.array((x.flatten(), y.flatten(), z.flatten())).T
    velocities = np.random.uniform(0, 0, size=(point_count_w * point_count_h, 3))

    # Создадим точки
    points = [MaterialPointClass(positions[i], velocity=velocities[i], mass=M) for i in
              range(point_count_w * point_count_h)]

    for i in range(point_count_w // 3):
        points[i * point_count_h].fixed = True
        points[(point_count_w - i - 1) * point_count_h].fixed = True


    # Создадим пружины
    for i in range(point_count_w):
        for j in range(point_count_h):
            if j < point_count_h - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + 1], K)
            if i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h], K)
            if j < point_count_h - 1 and i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h + 1], K)
            if j > 0 and i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h - 1], K)

    return positions, points

In [ ]:
def create_rect_cloth_h(point_count_w, point_count_h, MaterialPointClass):
    """
    Горизонтально падающая ткань, подвешанная с обоих сторон за point_count_w/3 вершин
    """
    x_ = np.linspace(0.25, 0.75, point_count_w)
    y_ = np.linspace(1., 1., 1)
    z_ = np.linspace(0.5, 0.0, point_count_h)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    z, y = y, z

    positions = np.array((x.flatten(), y.flatten(), z.flatten())).T
    velocities = np.random.uniform(0, 0, size=(point_count_w * point_count_h, 3))

    # Создадим точки
    points = [MaterialPointClass(positions[i], velocity=velocities[i], mass=M) for i in
              range(point_count_w * point_count_h)]

    for i in range(point_count_w // 3):
        points[i * point_count_h].fixed = True
        points[(point_count_w - i - 1) * point_count_h].fixed = True


        # Создадим пружины
    for i in range(point_count_w):
        for j in range(point_count_h):
            if j < point_count_h - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + 1], K)
            if i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h], K)
            if j < point_count_h - 1 and i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h + 1], K)
            if j > 0 and i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h - 1], K)

    return positions, points

Разные глобальные силы, через подобные функции можем, например, создать ветер!

In [ ]:
def get_just_gravity(time):
    def global_forces(pos, mass):
        return GLOBAL_CONSTANT_FORCE * mass
    return global_forces

Довольно проклятая функция анимации. Мог бы сделать её по-приличнее но не стал.

In [ ]:
# Инициализация графика. Возвращает акторов в которых пишем точки
animation_time = 0
def init_plot(pos):
    global connections, animation_time
    connections = []
    animation_time = 0

    global fig, ax
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    ax.set(xlim3d=(0, 1), xlabel='X')
    ax.set(ylim3d=(0, 1), ylabel='Y')
    ax.set(zlim3d=(0, 1), zlabel='Z')

    p3 = ax.plot(positions.T[0], positions.T[1], positions.T[2], 'o', c='green')
    return p3[0]


# Отрисовка связей
def handle_connections(pts, pos, ax):
    global connections

    for c in connections:
        c[0].remove()
    connections = []
    _connections = []

    for i in range(len(pos)):
        for p in pts[i]:
          _connections.extend([(p.position, i["other"].position) for i in p.connections])

    for c in _connections:
        connections.append(ax.plot([c[0][0], c[1][0]], [c[0][1], c[1][1]], [c[0][2], c[1][2]], c='red'))

rotation = 0
rotation_speed = 0.05
def update(frame, pts, pos, plot, ax, get_forces_from_time) -> None:
    global animation_time, rotation
    animation_time += DT

    # Вращаем камеру чтоб было красиво
    rotation += rotation_speed
    ax.azim = rotation

    # Собственно итерация физики точек
    for j in range(SKIPS):
        for i in range(len(pos)):
            for p in pts[i]:
                p.update(DT, get_forces_from_time(animation_time))
                p.frame = frame

    for i in range(len(pos)):
        plot[i].set_data(pos[i].T[:2])
        plot[i].set_3d_properties(pos[i].T[2], 'z')

    # Отрисовка соединений. Реализация очень убогая
    # SHOW_CONNECTIONS = False ускоряет рендер в несколько раз
    if SHOW_CONNECTIONS:
        handle_connections(pts, pos, ax)

    # Проецентный индикатор отрисовки
    if int(frame * 10 / FRAMES) < int((frame + 1) * 10 / FRAMES):
        print(f"{round(frame * 10 / FRAMES) * 10}% done")
    return

Сложно в это поверить, но мы готовы по-рендерить анимацию! Запустим простой тест на падающей ткани.

*Note: некоторые анимации тратят очень много времени! Если хочется посмотреть на результаты можно либо скипнуть рендеринг доли фреймов через SKIPS и поставить SHOW_CONNECTIONS = False, значительно ускорив рендеринг.*

*Так же в конце каждого блока анимации вставил эмбед с i.imgur.com, правда не уверен как долго они там пролежат*

*Альтернативно - в конце ноутбука есть ссылки на все срендеренные гифы на диске*

In [ ]:
SHOW_CONNECTIONS = True   # Флаг отрисовки пружин
SKIPS = 10                 # Ускоряем рендеринг собирая только 1 из SKIPS кадров
FRAMES = 100             # Количество итоговых фреймов
rotation = 45             # Стартовая позиция камеры
rotation_speed = 0.05     # Финальная позиция

In [ ]:
positions, points = create_rect_cloth_h(8, 8, MaterialPointVerletWithSpring)
points_graph = init_plot(positions)
ani = animation.FuncAnimation(
    fig, update, FRAMES, fargs=([points],
                                [positions],
                                [points_graph],
                                ax, get_just_gravity),
                                interval=int(DT * SKIPS * 1000),
                                blit=False)

"""Сохраним анимацию"""

writer = animation.PillowWriter(fps = 30, bitrate=1800)
ani.save(f'Falling_clothes.gif', writer=writer)

![](https://i.imgur.com/o9ZLiPd.gif)

Чудестно! Пользуясь случаем давай-те попробуем поиграть с симуляцией...

In [ ]:
!pip install perlin-noise

In [ ]:
from perlin_noise import PerlinNoise
noise = PerlinNoise()

Сейчас попробуем просимулировать ветер дующий вдоль Y.

Для рандомности будем брать константу + perlinNoise по координатам х, у и времени симуляции, чтобы ветер был неравномерный но неприрывный!

(это я играюсь если что, на баллы за это задание не претендую)

In [ ]:
def get_gravity_and_wind(time):
    def global_forces(pos, mass):
        wind_power = noise([time, pos[0], pos[1]]) + 0.4
        force = np.array([0, wind_power, 0], dtype=np.float64)
        return force + GLOBAL_CONSTANT_FORCE * mass
    return global_forces

In [ ]:
SHOW_CONNECTIONS = True  # Флаг отрисовки пружин
SKIPS = 1                # Ускоряем рендеринг собирая только 1 из SKIPS кадров
FRAMES = 1000            # Количество итоговых фреймов
rotation_speed = 0.1     # Скорость вращения камеры
rotation = 20            # Стартавая позиция камеры

In [ ]:
positions, points = create_rect_cloth_w(8, 8, MaterialPointVerletWithSpring)
points_graph = init_plot(positions)
ani = animation.FuncAnimation(
    fig, update, FRAMES, fargs=([points],
                                [positions],
                                [points_graph],
                                ax, get_gravity_and_wind),
                                interval=int(DT * SKIPS * 1000),
                                blit=False)

"""Сохраним анимацию"""

writer = animation.PillowWriter(fps=30, bitrate=1800)
ani.save(f'Clothes_with_wind.gif', writer=writer)

![](https://i.imgur.com/TYrEIf9.gif)

так, а что там с-
# Симуляция ткани связанной в цилиндр в 3д
так вот же она!

In [ ]:
def create_cil_cloth(point_count_w, point_count_h, MaterialPointClass):
    """
    Вертикально подвешанный за все верхние точки целиндр
    """
    x_ = np.linspace(0., ((point_count_w - 1) / point_count_w) * 2 * np.pi, point_count_w)
    y_ = np.linspace(1, 1, 1)
    z_ = np.linspace(1., 0.50, point_count_h)

    x__, y__, z = np.meshgrid(x_, y_, z_, indexing='ij')

    x = np.sin(x__) / 4 + 0.5
    y = np.cos(x__) / 4 + 0.5

    positions = np.array((x.flatten(), y.flatten(), z.flatten())).T
    velocities = np.random.uniform(0, 0, size=(point_count_w * point_count_h, 3))

    # Создадим точки
    points = [MaterialPointClass(positions[i], velocity=velocities[i], mass=M) for i in
              range(point_count_w * point_count_h)]

    for i in range(point_count_w):
        points[point_count_h * i].fixed = True

    # Создадим пружины
    for i in range(point_count_w):
        for j in range(point_count_h):
            if j < point_count_h - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + 1], K)
            if i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h], K)
            else:
                points[j + i * point_count_h].connect(points[j], K)
            if j < point_count_h - 1 and i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h + 1], K)
            elif j < point_count_h - 1:
                points[j + i * point_count_h].connect(points[j + 1], K)
            if j > 0 and i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h - 1], K)
            elif j > 0:
                points[j + i * point_count_h].connect(points[j - 1], K)


    return positions, points

In [ ]:
SHOW_CONNECTIONS = True  # Флаг отрисовки пружин
SKIPS = 1                # Ускоряем рендеринг собирая только 1 из SKIPS кадров
FRAMES = 100             # Количество итоговых фреймов
rotation_speed = 0.5     # Скорость вращения камеры
rotation = 0             # Стартавая позиция камеры

In [ ]:
positions, points = create_cil_cloth(15, 8, MaterialPointVerletWithSpring)
points_graph = init_plot(positions)
ani = animation.FuncAnimation(
    fig, update, FRAMES, fargs=([points],
                                [positions],
                                [points_graph],
                                ax, get_just_gravity),
                                interval=int(DT * SKIPS * 1000),
                                blit=False)

"""Сохраним анимацию"""

writer = animation.PillowWriter(fps=30, bitrate=1800)
ani.save(f'Clothes_in_cilinder.gif', writer=writer)

![](https://i.imgur.com/yqQHLsv.gif)

Пока она просто висит - не особо интересно наблюдать. Давайте накиним на её вершины стартовую скорость!

In [ ]:
def create_cil_cloth_with_a_push(point_count_w, point_count_h, MaterialPointClass):
    """
    Вертикально подвешанный за все верхние точки целиндр
    """
    x_ = np.linspace(0., ((point_count_w - 1) / point_count_w) * 2 * np.pi, point_count_w)
    y_ = np.linspace(1, 1, 1)
    z_ = np.linspace(1., 0.50, point_count_h)

    x__, y__, z = np.meshgrid(x_, y_, z_, indexing='ij')

    x = np.sin(x__) / 4 + 0.5
    y = np.cos(x__) / 4 + 0.5

    positions = np.array((x.flatten(), y.flatten(), z.flatten())).T

    x_ = np.linspace(0., ((point_count_w - 1) / point_count_w) * 2 * np.pi, point_count_w)
    y_ = np.linspace(1, 1, 1)
    z_ = np.linspace(1., 0.50, point_count_h)

    x__, y__, z = np.meshgrid(x_, y_, z_, indexing='ij')

    """
    Добавим вектор скорости внутрь поверхности в оси XY
    """
    x = np.sin(x__)
    y = np.cos(x__)

    velocities = np.array((x.flatten(), y.flatten(), z.flatten() * 0)).T

    # Создадим точки
    points = [MaterialPointClass(positions[i], velocity=velocities[i], mass=M) for i in
              range(point_count_w * point_count_h)]

    for i in range(point_count_w):
        points[point_count_h * i].fixed = True

    # Создадим пружины
    for i in range(point_count_w):
        for j in range(point_count_h):
            if j < point_count_h - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + 1], K)
            if i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h], K)
            else:
                points[j + i * point_count_h].connect(points[j], K)
            if j < point_count_h - 1 and i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h + 1], K)
            elif j < point_count_h - 1:
                points[j + i * point_count_h].connect(points[j + 1], K)
            if j > 0 and i < point_count_w - 1:
                points[j + i * point_count_h].connect(points[j + i * point_count_h + point_count_h - 1], K)
            elif j > 0:
                points[j + i * point_count_h].connect(points[j - 1], K)


    return positions, points

In [ ]:
SHOW_CONNECTIONS = True  # Флаг отрисовки пружин
SKIPS = 1                # Ускоряем рендеринг собирая только 1 из SKIPS кадров
FRAMES = 500             # Количество итоговых фреймов
rotation_speed = 0.1     # Скорость вращения камеры
rotation = 0             # Стартавая позиция камеры

In [ ]:
positions, points = create_cil_cloth_with_a_push(15, 8, MaterialPointVerletWithSpring)

points_graph = init_plot(positions)
ani = animation.FuncAnimation(
    fig, update, FRAMES, fargs=([points],
                                [positions],
                                [points_graph],
                                ax, get_just_gravity),
                                interval=int(DT * SKIPS * 1000),
                                blit=False)

"""Сохраним анимацию"""

writer = animation.PillowWriter(fps=30, bitrate=1800)
ani.save(f'Clothes_in_cilinder_with_a_push.gif', writer=writer)

![](https://i.imgur.com/MiDp2TD.gif)

Деформируется и возвращается обратно! Как и ожидалось собственно.

Последнее - цилиндр с ветром

In [ ]:
SHOW_CONNECTIONS = True  # Флаг отрисовки пружин
SKIPS = 1                # Ускоряем рендеринг собирая только 1 из SKIPS кадров
FRAMES = 500             # Количество итоговых фреймов
rotation_speed = 0.1     # Скорость вращения камеры
rotation = 0             # Стартавая позиция камеры

In [ ]:
positions, points = create_cil_cloth(15, 8, MaterialPointVerletWithSpring)
points_graph = init_plot(positions)
ani = animation.FuncAnimation(
    fig, update, FRAMES, fargs=([points],
                                [positions],
                                [points_graph],
                                ax, get_gravity_and_wind),
                                interval=int(DT * SKIPS * 1000),
                                blit=False)

"""Сохраним анимацию"""

writer = animation.PillowWriter(fps=30, bitrate=1800)
ani.save(f'Clothes_in_cilinder_with_wind.gif', writer=writer)

![](https://i.imgur.com/8Tle7ja.gif)

Красиво, доволен, иду отдыхать

[Ссылки на срендеренные гифы](https://drive.google.com/drive/folders/14h7AI7rOHuNCUPFm20T0-gsi-_ulbUu9?usp=sharing)
